<a href="https://colab.research.google.com/github/dlskawns/cp2/blob/main/Clustering_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정책 데이터셋 샘플 수 축소를 위한 클러스터링

### 모듈 불러오기

In [ ]:
!pip install category_encoders
!pip install kmodes

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, l1_l2
from tqdm import tqdm
import pickle
import numpy as np
from tqdm import tqdm
from category_encoders import OrdinalEncoder
from kmodes.kprototypes import KPrototypes
import copy


import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 불러오기

In [ ]:
import pickle

# 태깅 추출된 정책 데이터셋 불러오기
with open('/content/drive/MyDrive/웰로/data/our_final0117.pkl', 'rb') as f:
    data1 = pickle.load(f)
# 태깅 추출된 유저 데이터셋 불러오기
with open('/content/drive/MyDrive/웰로/data/user_final0117 (1).pkl', 'rb') as f:
    user1 = pickle.load(f)
# 추천모델A 를 전체 user에 적용해 1차적으로 정책 list sorting (약 39000여개 index list)
with open('/content/drive/MyDrive/웰로/data/idx_lst.pkl', 'rb') as f:
  idx_lst = pickle.load(f)

In [ ]:

class policy_clustering():
  """
  정책 데이터셋의 수를 줄이기 위해 군집화를 하는 클래스
  정책 데이터 태그를 전처리하고, categorical, numerical data를 군집화 할 수 있는 k-prototypes 클러스터링 진행
  """
  
  def __init__(self, df, idx_lst):
    self.df = df.copy()
    self.df = self.df.iloc[idx_lst].reset_index()
    
    # 필요한 feature만 가져옴
    self.df = self.df[['정책ID', '대상연령시작', '대상연령끝', '시도', '중위소득', '관심상황특성', '대상특성', '대상특성상세', 'MB_15', '소관기관유형', '지원유형', '지원유형상세',
       '신청절차', '성별', '결혼', '학력', '가구원', '자녀', '자녀상세', '직장', '직장상세']]
    
    # 취업지원이 두 가지로 나뉘어져 있어 하나로 통합
    self.df['MB_15'] = self.df['MB_15'].map(lambda x: x.replace('취업지원', '취업 지원'))

    # 이혼&재혼 -> 기혼
    self.df['결혼'] = self.df['결혼'].map(lambda x: x.replace('이혼', '기혼').replace('재혼','기혼').replace('기혼,미혼','무관'))

    # 기존 샘플 중 직장, 가구원, 결혼에서 무관이 아닌 샘플 추출
    sample_1 = list(self.df[(self.df['직장']!='무관')&(self.df['가구원']!='무관')].index.values)
    sample_2 = list(self.df[self.df['가구원']!='무관'].index.values)
    sample_3 = list(self.df[self.df['결혼']!='무관'].index.values)
    self.sample_n = list(set(sample_1 + sample_2 + sample_3))

    self.numeric_cols = ['정책ID','대상연령시작','대상연령끝','max_income','min_income'] # 연속형 특성 
    self.ordinal_cols = ['시도','소관기관유형','지원유형','성별','결혼','자녀']          # 범주형 특성
    self.other_cols = list()  # 전체 원핫으로 바꾼것 포함 범주형 특성
  
  def class_to_columns(self, df, col):
    """
    feature를 one-hot 인코딩 해주는 함수
    중복 태그의 문제를 해결하기 위함
    """
    b = []
    for i in df[col]:
      if ',' in i:
        i = i.split(',')
        b.extend(i)
      else:
        b.append(i) 
    c = list(set(b))
    for i in c:
      df[i] = 0
    for i in tqdm(range(len(df))):
      for j in c:
        if j in df[col][i]:
          df[j][i] =1

    df=df.drop(columns = col)

  def class_to_columns_two(self,df, col):
    """
    대상특성, 직장, 지원유형 등의 어느정도 필요하면서 중복이 많은 특성은 클러스터링 시 one hot 인코딩 후 진행할 것.
    클러스터링 속성 지정을 위해 함수 재설정
    """
    b = []
    for i in df[col]:
      if ',' in i:
        i = i.split(',')
        b.extend(i)
      else:
        b.append(i) 
    c = list(set(b))
    for i in c:
      df[col+i] = 0
    for i in tqdm(range(len(df))):
      for j in c:
        if j in df[col][i]:
          df[col+j][i] =1

  def income_change(self, df):
    """
    중위소득을 min, max로 나누어 파악할 수 있도록 하는 함수
    ex: 0,10,20,30,40 -> min_income = 0, max_income = 40
    """

    df['중위소득'] = df['중위소득'].fillna('')
    b = []
    df['max_income'] = None
    df['min_income'] = None
    for i in tqdm(range(len(df['중위소득']))):
      if ',' in df['중위소득'][i]:
        if '기초생활수급자' in df['중위소득'][i]:
          a = df['중위소득'][i].split(',')
          a.remove('기초생활수급자')
          a = list(set(map(int, a)))
          max_income = max(a)
          df['max_income'][i] = max_income
          df['min_income'][i] = 0
        else:
          a = df['중위소득'][i].split(',')
          a = list(set(map(int, a)))
          max_income = max(a)
          min_income = min(a)
          df['max_income'][i] = max_income
          df['min_income'][i] = min_income
      elif df['중위소득'][i] =='기초생활수급자':
        df['max_income'][i] = 30
        df['min_income'][i] = 0
      else:
        df['max_income'][i] = 200
        df['min_income'][i] = 0

  def preprocessing_feature(self):
    """
    정책 데이터의 각각의 feature를 User tag와 비교할 수 있도록 전처리하는 함수
    중요 tag(MB_15, 대상특성, 직장, 지원유형 등의 태그를 포함)
    """
    print('MB_15 feature 해체 진행 \n1/5 진행중')
    self.class_to_columns(self.df, 'MB_15')
    print('중위소득 feature 변경 진행 \n2/5 진행중')
    self.income_change(self.df)
    print('대상특성 feature 해체 진행 \n3/5 진행중')
    self.class_to_columns_two(self.df, '대상특성')
    print('직장 feature 해체 진행 \n4/5 진행중')
    self.class_to_columns_two(self.df, '직장')
    print('지원유형 feature 해체 진행 \n5/5 진행중')
    self.class_to_columns_two(self.df, '지원유형')
    self.df = self.df.drop(columns = ['중위소득','대상특성상세','관심상황특성','직장','지원유형상세','신청절차','학력','가구원','자녀상세','직장상세','대상특성','MB_15'])
    self.other_cols = list(self.df.drop(columns = self.numeric_cols).columns.values)  # 전체 원핫으로 바꾼것 포함 범주형 특성

  def clustering_data(self):
    """
    Ordinal Encoding하여 categorical features를 수치화 한 뒤, 클러스터링을 진행하는 함수
    사전에 Elbow Methods를 통해 최적의 군집개수 확인 k=5
    """
    enc = OrdinalEncoder(cols = self.ordinal_cols)
    enced_data = enc.fit_transform(self.df)
    cat_col_cluster = list(range(0,78))
    for i in [1,2,20,21]:
      cat_col_cluster.remove(i)
    
    print('클러스터링 진행')
    kproto = KPrototypes(n_clusters=5, max_iter = 3, n_jobs= 4, init='Cao')
    kproto.fit_predict(enced_data, categorical=cat_col_cluster)
    
    self.df['cluster'] = kproto.labels_

  def clustered_idx(self):
    """
    군집화 완료된 후, 적절한 군집 내 샘플 500개 및 필요 샘플 500개로 총 1000개의 정책 데이터 샘플을 추출
    """
    data = self.df.drop(self.df.iloc[self.sample_n].index)
    data_0 = list(data[data['cluster']==0].sample(100, random_state = 42).index.values)
    data_1 = list(data[data['cluster']==1].sample(100, random_state = 42).index.values)
    data_2 = list(data[data['cluster']==2].sample(101, random_state = 42).index.values)
    data_3 = list(data[data['cluster']==3].sample(100, random_state = 42).index.values)
    data_4 = list(data[data['cluster']==4].sample(101, random_state = 42).index.values)

    data_lst = data_0+data_1+data_2+data_3+data_4+self.sample_n
    return data_lst


In [ ]:
class preprocessing_label():
  """
  policy_clustering을 통해 추출한 정책 데이터 샘플 list를 바탕으로
  임의의 유저 3000명 대상 조건 부합여부에 따라 labeling(1,0) 하기 위한 클래스
  """
  def __init__(self, user, data, idx_lst, data_lst):
    self.user = user.copy()
    # 정책 데이터셋 feature engineering
    self.loader = policy_clustering(data, idx_lst)
    self.data = self.loader.df.iloc[data_lst].reset_index()
    print('정책 데이터셋 중위소득 feature 변경 진행 \n1/3 진행중')
    self.loader.income_change(self.data)
    self.data['대상특성'] = self.data['대상특성'].map(lambda x: x.replace(' ', ''))
    print('정책 데이터셋 대상특성 feature 해체 진행 \n2/3 진행중')
    self.loader.class_to_columns(self.data, '대상특성')
    print('정책 데이터셋 MB_15 feature 해체 진행 \n3/3 진행중')
    self.loader.class_to_columns(self.data, 'MB_15')

    self.data = self.data.drop(columns = ['중위소득','대상특성상세','관심상황특성','지원유형상세','신청절차','가구원','자녀상세','직장상세','대상특성','MB_15', ''])

    self.data = self.data[['정책ID', '대상연령시작', '대상연령끝', '시도', '소관기관유형', '지원유형', '학력', '성별', '결혼','자녀', '직장', 'max_income', 'min_income', 
                           '질병/부상/질환자', '국가유공자','한부모가정/조손가정',  '다자녀가정',  '해당사항없음', '다문화가족', '북한이탈주민', '농축수산인', '장애인', # '저소득층','독거노인','소년소녀가장','구호구제대상자',
                           '개인금융지원', '문화생활 지원', '취업 지원', '창업 지원', '교육지원(8~19세)','의료 지원', '보육지원(0~7세)', '주택-부동산 지원', '성인교육지원', '기업금융지원', '근로자 지원']]
    # 유저 데이터셋 feature engineering
  def user_preprocessing(self):
    # self.user = user.copy()
    self.user = self.user.drop(index = self.user[self.user['mb_15'].isnull()].sample(frac=0.9).index.values)
    self.user = self.user.drop(index = self.user[self.user['mb_12'].isnull()].sample(frac=0.9).index.values).reset_index()
    print('유저 데이터셋 대상특성 feature 해체 진행 \n1/3 진행중')
    self.loader.class_to_columns(self.user, 'mb_10+대상특성')
    print('유저 데이터셋 중위소득 feature 변경 진행 \n2/3 진행중')
    self.income_change_user()
    print('유저 데이터셋 MB_15 feature 해체 진행 \n3/3 진행중')
    self.user['mb_15'].fillna('관심정책없음',inplace=True)
    self.loader.class_to_columns(self.user, 'mb_15')

    self.user = self.user.drop(columns = ['시군구','자녀상세','자녀수','자녀','mb_10','mb_11','mb_10+대상특성', 'mb_11+관심상황특성','mb_12','mb_13','mb_14','mb_15','index','저소득층','독거노인','소년소녀가장','구호구제대상자'])
    self.user['결혼'] = self.user['결혼'].map(lambda x: x.replace('미혼,기혼','무관'))



  def income_change_user(self):
    self.user['mb_12'] = self.user['mb_12'].fillna('')
    b = []
    self.user['max_income'] = None
    self.user['min_income'] = None
    for i in tqdm(range(len(self.user['mb_12']))):
      if '이하' in self.user['mb_12'][i]:
        self.user['max_income'][i] = 40
        self.user['min_income'][i] = 0
      elif '이상' in self.user['mb_12'][i]:
        self.user['max_income'][i] = 110     # 100이상은 모두 포함
        self.user['min_income'][i] = 100
      elif '기초생활수급자' in self.user['mb_12']:
        self.user['max_income'][i] = 30
        self.user['min_income'][i] = 0
      elif '사이' in self.user['mb_12'][i]:
        if '40~60' in self.user['mb_12'][i]:
          self.user['max_income'][i] = 60
          self.user['min_income'][i] = 40
        elif '60~80' in self.user['mb_12'][i]:
          self.user['max_income'][i] = 80
          self.user['min_income'][i] = 60
        elif '80~100' in self.user['mb_12'][i]:
          self.user['max_income'][i] = 100
          self.user['min_income'][i] = 80
        else: # 40% 이하
          self.user['max_income'][i] = 40
          self.user['min_income'][i] = 0
      else:
        self.user['max_income'][i] = 110     # 100이상은 모두 포함
        self.user['min_income'][i] = 0


  def data_creating(self):
    train = pd.DataFrame()
    user2 = self.user.sample(frac=1)[:3000]
    user2['label'] = 0
    for i in tqdm(range(0, 3000)):
      a = pd.DataFrame([user2.iloc[i]]*1000)
      a['정책ID'] = self.data['정책ID'].values
      c = pd.merge(a,self.data,on='정책ID')
      # labeling_one(c)
      train = pd.concat([train, c], axis = 0)
    return train
  

In [ ]:
# from pandas.core.base import NoNewAttributesMixin
def labeling_one(df):
  """
  preprocess_label 클래스에서 사용할 조건 비교에 따른 라벨부여 함수
  """
  df['label'] = None
  df = df.reset_index()
  for i in tqdm(range(len(df))):
    # MB15
    s = 0
    if (df['기업금융지원_x'][i]== 1 and df['기업금융지원_y'][i] == 1) \
    or (df['교육지원(만8~19세)'][i] == 1 and df['교육지원(8~19세)'][i] == 1) \
    or (df['보육지원(만0~7세)'][i] == 1 and df['보육지원(0~7세)'][i] == 1) \
    or (df['취업 지원_x'][i] ==1 and df['취업 지원_y'][i]==1)\
    or (df['창업 지원_x'][i] == 1 and df['창업 지원_y'][i] == 1) \
    or (df['개인금융지원_x'][i] == 1 and df['개인금융지원_y'][i] == 1) \
    or (df['문화생활 지원_x'][i] == 1 and df['문화생활 지원_y'][i] == 1) \
    or (df['성인교육지원_x'][i] == 1 and df['성인교육지원_y'][i] == 1) \
    or (df['의료 지원_x'][i] == 1 and df['의료 지원_y'][i] == 1) \
    or (df['의료 지원_x'][i] == 1 and df['의료 지원_y'][i] == 1) \
    or (df['주택-부동산 지원_x'][i] == 1 and df['주택-부동산 지원_y'][i] == 1)\
    or df['관심정책없음'][i] == 1:
      s+=1
    # # 중위소득  -> 중위소득 관련 관계 정의를 제외하고 진행,  조건 설정 시 실제 추천 성능 개선될 것으로 예상.
    # if ((df['max_income_x'][i] =< df['max_income_y']) \ 
    # and (df['min_income_x'][i] => df['max_income_y'])) \
    # s+=1
    #성별
    if df['성별_x'][i] == df['성별_y'][i] or df['성별_y'][i] =='무관':
      s += 1
    # 결혼
    if df['결혼_x'][i] == df['결혼_y'][i] or df['결혼_y'][i] == '무관':
      s += 1
    # 직장
    if df['직장_x'][i] in df['직장_y'][i] or df['직장_y'][i] == '무관':
      s += 1
    # 학력
    if df['학력_x'][i] in df['학력_y'][i] or df['학력_y'][i] == '무관':
      s += 1

    if df['국가유공자_x'][i] == df['국가유공자_y'][i] \
    and df['다자녀가정_x'][i] == df['다자녀가정_y'][i] \
    and df['한부모가정/조손가정_x'][i] == df['한부모가정/조손가정_y'][i] \
    and df['다문화가족_x'][i] == df['다문화가족_y'][i]  \
    and df['질병/부상/질환자_x'][i] == df['질병/부상/질환자_y'][i]  \
    and df['장애인_x'][i] == df['장애인_y'][i]  \
    and df[' 농축수산인'][i] == df['농축수산인'][i]  \
    and df['북한이탈주민_x'][i] == df['북한이탈주민_y'][i]  \
    and df['해당사항없음_x'][i] == df['해당사항없음_y'][i]:
      s += 1
    if s >= 5:
      df['label'][i] = 1
    else:
      df['label'][i] = 0

  return df


In [ ]:
# 클러스터링 진행위한 정책 데이터셋 전처리
feat = policy_clustering(data1, idx_lst)
feat.preprocessing_feature()

sample_n 개수 498
MB_15 feature 해체 진행 
1/5 진행중


100%|██████████| 39340/39340 [00:31<00:00, 1268.07it/s]


중위소득 feature 변경 진행 
2/5 진행중


100%|██████████| 39340/39340 [00:34<00:00, 1141.37it/s]


대상특성 feature 해체 진행 
3/5 진행중


100%|██████████| 39340/39340 [00:25<00:00, 1527.07it/s]


직장 feature 해체 진행 
4/5 진행중


100%|██████████| 39340/39340 [00:17<00:00, 2208.73it/s]


지원유형 feature 해체 진행 
5/5 진행중


100%|██████████| 39340/39340 [00:27<00:00, 1422.86it/s]


In [ ]:
# 정책 데이터셋 클러스터링 진행
feat.clustering_data()

클러스터링 진행


In [ ]:
# 클러스터링 후 각 군집별 100개씩 sample 인덱스 추출
data_lst = feat.clustered_idx()

In [ ]:
# 유저 + 정책 데이터셋 라벨 설정 인스턴스 생성
lab = preprocessing_label(user1, data1, idx_lst, data_lst)
lab.user_preprocessing()

sample_n 개수 498
정책 데이터셋 중위소득 feature 변경 진행 
1/3 진행중


100%|██████████| 1000/1000 [00:00<00:00, 1857.64it/s]


정책 데이터셋 대상특성 feature 해체 진행 
2/3 진행중


100%|██████████| 1000/1000 [00:00<00:00, 1900.22it/s]


정책 데이터셋 MB_15 feature 해체 진행 
3/3 진행중


...

100%|██████████| 14077/14077 [00:05<00:00, 2559.09it/s]


유저 데이터셋 중위소득 feature 변경 진행 
2/3 진행중


100%|██████████| 14077/14077 [00:09<00:00, 1453.36it/s]


유저 데이터셋 MB_15 feature 해체 진행 
3/3 진행중


100%|██████████| 14077/14077 [00:13<00:00, 1042.48it/s]


In [ ]:
# 유저 + 정책 데이터셋 merge
last_data = lab.data_creating()

100%|██████████| 3000/3000 [17:34<00:00,  2.85it/s]


In [ ]:
l = labeled_data.copy()

In [ ]:
# 유저, 정책 조건에 따른 라벨 생성
l = labeling_one(l)

100%|██████████| 3000000/3000000 [7:25:01<00:00, 112.35it/s]


In [ ]:
# import pickle

# with open('0214.pkl' , 'wb') as f:
#   pickle.dump(l, f, pickle.HIGHEST_PROTOCOL)